# Feature elimination

In [11]:
import sys
sys.path.insert(0, '/home/pavel/Documents/0Research/Projects/LANL-Earthquake/src/data_processing')
from dp_utils import *
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
data_path = '/home/pavel/Documents/0Research/Projects/LANL-Earthquake/data/EQs/EQ_2.h5'
df = pd.read_hdf(data_path, key = 'table')
df.columns = ['s','ttf']

In [12]:
config_path = "/home/pavel/Documents/0Research/Projects/LANL-Earthquake/src/data_processing/dp_config.json"
with open(config_path, 'rb') as config:
    params = json.load(config)

routines = params['routines']
default_window_size = params['window_size']
new_df = process_df(df, routines,default_window_size)

w_abs_energy(s, window_size=5000): 100%|██████████| 8886/8886 [00:00<00:00, 16840.63it/s]
	 window decorator: 
	 - window size: 5000
w_absolute_sum_of_changes(s, window_size=15000): 100%|██████████| 2962/2962 [00:00<00:00, 9566.04it/s]
	 window decorator: 
	 - window size: 15000
w_autocorrelation(s, window_size=10000, lag=100): 100%|██████████| 4443/4443 [00:01<00:00, 3525.75it/s]
	 window decorator: 
	 - window size: 10000
w_binned_entropy(w_absolute_sum_of_changes(s, window_size=15000), window_size=2, max_bins=10): 100%|██████████| 1481/1481 [00:00<00:00, 5108.16it/s]
	 window decorator: 
	 - window size: 2
w_c3(s, window_size=10000, lag=100): 100%|██████████| 4443/4443 [00:00<00:00, 8060.37it/s]
	 window decorator: 
	 - window size: 10000
w_psd(s, window_size=10000, fs=4000000.0): 100%|██████████| 4443/4443 [00:02<00:00, 2174.89it/s]
	 window decorator: 
	 - window size: 10000
w_savgol_filter(w_psd(s, window_size=10000, fs=4000000.0), window_size=9999999999, window_length=201, polyo

In [13]:
save_to = '/home/pavel/Documents/0Research/Projects/LANL-Earthquake/data/featurized_data_EQ_2.h5'
new_df.to_hdf(save_to, key = 'table')

Preparing the data

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X = new_df.loc[:, new_df.columns != 'ttf']
X = np.array(X)
scaler.fit(X)
X = scaler.transform(X)
y = np.array(new_df.ttf)

# Removing features with low variance

In [15]:
from sklearn.feature_selection import VarianceThreshold
print("было: ",len(new_df.columns))
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
selected_features = sel.fit_transform(new_df)
print("стало: ",selected_features.shape[1])

было:  8
стало:  6


# Univariate feature selection

Univariate feature selection works by selecting the best features based on univariate statistical tests. It can be seen as a preprocessing step to an estimator.

**SelectKBest** removes all but the  highest scoring features.

The methods based on **F-test** estimate the degree of linear dependency between two random variables. On the other hand, **mutual information** methods can capture any kind of statistical dependency, but being nonparametric, they require more samples for accurate estimation.

In [19]:
from sklearn.feature_selection import SelectKBest,SelectFpr, SelectFdr
from sklearn.feature_selection import f_regression, mutual_info_regression
X_new = SelectKBest(mutual_info_regression, k=3).fit_transform(X, y)
print(X_new.shape)

(4443, 3)


**SelectFpr**

Filter: Select the pvalues below alpha based on a FPR test.

FPR test stands for False Positive Rate test. It controls the total amount of false detections.

In [20]:
X_new = SelectFpr(f_regression,alpha = 0.1).fit_transform(X, y)
print(X_new.shape)

(4443, 5)


/home/pavel/.local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/home/pavel/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/pavel/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/pavel/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/home/pavel/.local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:560: RuntimeWarning: invalid value encountered in less
  return self.pvalues_ < self.alpha


**SelectFdr**

Filter: Select the p-values for an estimated false discovery rate

This uses the Benjamini-Hochberg procedure. alpha is an upper bound on the expected false discovery rate.

In [21]:
X_new = SelectFdr(f_regression,alpha = 0.1).fit_transform(X, y)
print(X_new.shape)

(4443, 5)


/home/pavel/.local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:619: RuntimeWarning: invalid value encountered in less_equal
  np.arange(1, n_features + 1)]
/home/pavel/.local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:622: RuntimeWarning: invalid value encountered in less_equal
  return self.pvalues_ <= selected.max()


# Recursive feature elimination

Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), the goal of recursive feature elimination (**RFE**) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

In [22]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR

In [29]:
import sklearn
hasattr(sklearn,'tree.DecisionTreeRegressor')

False

In [9]:
estimator = DTR()
selector = RFE(estimator, 3, step=1)

selector = selector.fit(X, y)
print(selector.support_) 
print("Remaining features: ")
print([col for indicator, col in zip(selector.support_,list(new_df.columns)) if indicator == True])
print(selector.ranking_)

[ True  True False False False False  True]
Remaining features: 
['w_abs_energy(s, window_size=5000)', 'w_absolute_sum_of_changes(s, window_size=15000)', 'w_savgol_filter(w_psd(s, window_size=10000, fs=4000000.0), window_size=9999999999, window_length=201, polyorder=1)']
[1 1 2 5 3 4 1]


In [10]:
selector

RFE(estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
  n_features_to_select=3, step=1, verbose=0)

In [166]:
estimator = RFR()
selector = RFE(estimator, 3, step=1)

selector = selector.fit(X, y)
print(selector.support_) 
print("Remaining features: ")
print([col for indicator, col in zip(selector.support_,list(new_df.columns)) if indicator == True])
print(selector.ranking_)

[ True  True False False False False  True]
Remaining features: 
['w_abs_energy(df, window_size=5000)', 'w_absolute_sum_of_changes(df, window_size=15000)', 'w_savgol_filter(df, window_size=9999999999, window_length=201, polyorder=1)']
[1 1 3 5 2 4 1]


# L1-based feature selection

Linear models penalized with the L1 norm have sparse solutions: many of their estimated coefficients are zero. When the goal is to reduce the dimensionality of the data to use with another classifier, they can be used along with feature_selection.SelectFromModel to select the non-zero coefficients. In particular, sparse estimators useful for this purpose are the linear_model.Lasso for regression, and of linear_model.LogisticRegression and svm.LinearSVC for classification

In [161]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

print(X.shape)
regressor = Lasso(alpha = 0.0015, normalize = True).fit(X, y)
model = SelectFromModel(regressor, prefit=True)
X_new = model.transform(X)
print(X_new.shape)

(4443, 7)
(4443, 2)


# Tree-based feature selection

In [162]:
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel

In [163]:
print(X.shape)
clf = ExtraTreesRegressor(n_estimators=50)
clf = clf.fit(X, y)
model = SelectFromModel(clf, prefit=True)
print(clf.feature_importances_)
X_new = model.transform(X)
print(X_new.shape)

(4443, 7)
[0.16389555 0.35380295 0.11044488 0.         0.10723329 0.12514752
 0.13947582]
(4443, 2)
